# Clustering de Comentarios en Facebook

## Cargar el archivo
Se extrajeron 1,569 comentarios de septiembre y octubre año 2020 en los que se menciona a CitiBanamex

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD 
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import nltk
from mpl_toolkits.mplot3d import Axes3D
from nltk.corpus import stopwords
from wordcloud import WordCloud
plt.rcParams['figure.figsize'] = (16, 9)
#nltk.download('stopwords')
#nltk.download('punkt')
stop_words_sp = set(stopwords.words('spanish'))

In [ ]:
df = pd.read_csv('../data/BanamexFace.csv')
df.head()

#### Limpieza del texto (puntuación, tokenización y eliminado de stopwords en español)

In [ ]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stop_words_sp]
    return text

#### Creamos la Matriz Documento-Término (DTM) con puntuaciones TF-IDF

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(df['com'])

In [ ]:
X_features = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vect.get_feature_names())
X_features.head()

In [ ]:
X_features.info()

#### Removemos las columnas de términos redundantes y los que son poco frecuentes

In [ ]:
column = pd.DataFrame(X_features.sum())
column.head()

In [ ]:
X_features.drop(columns=['', 'abierta', 'abren', 'aclarar', 'alta', 'amado', 'amo', 'anualidaaaaaaaaaaaaad', 'anualidades', 'asco', 'audiomático', 'cajeros', 'cambiar', 'casi', 'cdmx', 'cobrando', 'convenio', 'créditos', 'debito', 'deja', 'desparcializar', 'enviar', 'hizo', 'inbox', 'liquidar', 'llega', 'martinez', 'mínimo', 'paso', 'plática', 'presentación', 'pude', 'puedan', 'quieres', 'quiten', 'regresen', 'responder', 'saludos', 'sola', 'sábado', 'targeta', 'tips', 'toño', 'transferir', 'transmisión', 'unan', 'ventanilla', 'viene', 'x'], inplace=True)

In [ ]:
stop = list(column[column[0]<2].index)

In [ ]:
len(stop)

In [ ]:
X_features.drop(columns=stop, inplace=True)

In [ ]:
X_features.info()

#### Aplicamos la descomposición SVD para poder trabajar con menos variables (10)

In [ ]:
truncatedSVD=TruncatedSVD(10)
X_svd = truncatedSVD.fit_transform(X_features)
X_svd[:10] 

#### Clustering utilizando KMeans

Identificamos la cantidad óptima de clusters

In [ ]:
Nc = range(1, 15)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X_svd).score(X_svd) for i in range(len(kmeans))]
score
plt.plot(Nc,score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

Generamos los centroides del Cluster

In [ ]:
kmeans = KMeans(n_clusters=6).fit(X_svd)
centroids = kmeans.cluster_centers_
print(centroids)

In [ ]:
centroids.shape

#### Visualizando los Clusters

In [ ]:
labels = kmeans.predict(X_svd)

C = kmeans.cluster_centers_
colores=['red','green','blue','cyan','yellow','purple']
asignar=[]
for row in labels:
    asignar.append(colores[row])

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X_svd[:, 0], X_svd[:, 1], X_svd[:, 2], c=asignar,s=60)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], marker='*', c=colores, s=1000)

Agregamos la marca de Cluster a cada comentario

In [ ]:
df['cluster'] = labels
df.head()

In [ ]:
df.cluster.value_counts()

Visualizamos wordclouds por cada segmento

In [ ]:
for i in range(6):
    text = " ".join([term.lower() for term in df.com[df['cluster']==i] if term not in stop_words_sp])
    plt.figure()
    wc= WordCloud(stopwords=stop_words_sp, background_color="white", colormap="Dark2", random_state=6)
    wc.generate(text)
    plt.imshow(wc)
    plt.axis("off")
    plt.title("Cluster " + str(i))
    plt.show()

#### Ahora podemos segmentar nuevos comentarios, utilizando el otro conjunto de datos

In [ ]:
dfs = pd.read_csv('../data/BanamexFaceNew.csv')
dfs.head()

Preparar los datos

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
S_tfidf = tfidf_vect.fit_transform(dfs['com'])

In [ ]:
S_features = pd.DataFrame(S_tfidf.toarray(), columns=tfidf_vect.get_feature_names())
S_features = S_features[X_features.columns]
S_features.info()

Aplicamos la misma reducción de dimensionalidad con SVD

In [ ]:
XS_svd = truncatedSVD.fit_transform(S_features)

Aplicamos el modelo

In [ ]:
slabels = kmeans.predict(XS_svd)
dfs['cluster'] = slabels
dfs.head(20)

In [ ]:
dfs.cluster.value_counts()

Elaborado por Luis Cajachahua bajo licencia MIT (2022)